<a href="https://colab.research.google.com/github/mcaires2/Projudi_Modificar_Sigilo_Documentos_Google_Colab/blob/master/SeleniumColab_Mofificar_Sigilo_C%C3%B3pia_P%C3%BAblica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparando o Google Colab para rodar o Selenium em headless mode, etapa obrigatória na primeira inicialização da máquina virtual Google Colab**

> Ao iniciar o Google Colab, ele abre uma instância (máquina virtual). Se você estiver iniciando esta máquina deve rodar a preparação do ambiente linux da célula a seguir para viabilizar a execução do Selenium via Google Colab



In [ ]:
!pip install selenium
from selenium import webdriver
from urllib.parse import quote
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


**Opções Disponíveis de Sigilo no Sistema Projudi:**


> Ao selecionar a opção correspodente você tem de usar o texto, respeitando maíuscula, mínuscula e acentuação programada no Projudi. Se achar mais conveniente, sugerimos copiar e copiar o texto da lista a seguir para evitar erros e encerramento da automação.



1.   Segredo
2.   Sigilo Mínimo
3.   Sigilo Médio
4.   Sigilo Intenso
5.   Sigilo Absoluto





In [ ]:

# Programação em Python + Google Colab + Selenium
# Objetivo é modificar o Nível de sigilo de Determinado Sequencial em um Processo Específico.
#



from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time




#
# I - Etapas de Ajustes pelo Usuário - Processo Nr, Sequencial,  Nível de Sigilo e SubItem .1 - Modificação Frequente
#
#





PROCESSO_NÚMERO = '0000288-84.2020.8.16.0014'   # nr processo entre aspas simples como no exemplo (númeração unica com pontos e traços)
SEQUENCIAL =22                                  # nr. sequencial sem aspas, campo numérico e inteiro exemplo: 20, 15, 14, 56, 35...
NIVEL_SIGILO = "Sigilo Médio"                        # opção sigilo dentre aquelas possíveis, formato texto e entre aspas; respeitar ortografia e acentuação projudi, vide explicação anotação texto célula anterior
SUBSEQUENCIAL_1_INCLUÍDO="S"                    # Anotar S para Sim ou N para Não. Por vezes, o sub item sequencial.1 é um documento que não deve ter o segredo modificado.




#
# II - Etapas de Ajustes pelo Usuário - login, senha e Vara_Juízo - Modificação Frequência Moderada, apenas quando trocar o usuário e juízo.
#
#



LOGIN = 'DIGITE AQUI SEU PERFIL DO PROJUDI'   # EX mslx.tec -> perfis de técnico ou analista judiciário
SENHA = 'DIGITE AQUI SUA SENHA'
VARA_JUIZO = '7ª Vara Cível de Londrina'      # na forma ortográfica informada pelo Projudi quando da etapa de login manual

#
# Fim do Setup







#
# III - A PARTIR DESTE PONTO NÃO MODIFICAR - AUTOMÁTICO 
#






driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)


# checar se a opção de sigilo escolhida pelo usuário é válida, caso negativo interrompe a execução do script

Controle_Opcao_Nivel_Sigilo =0

if NIVEL_SIGILO =="Segredo": Controle_Opcao_Nivel_Sigilo=1
if NIVEL_SIGILO =="Sigilo Mínimo": Controle_Opcao_Nivel_Sigilo=1
if NIVEL_SIGILO =="Sigilo Médio": Controle_Opcao_Nivel_Sigilo=1
if NIVEL_SIGILO =="Sigilo Intenso": Controle_Opcao_Nivel_Sigilo=1
if NIVEL_SIGILO =="Sigilo Absoluto": Controle_Opcao_Nivel_Sigilo=1

if Controle_Opcao_Nivel_Sigilo ==0: 
  print(f'Escolha do Nível de Sigilo "{NIVEL_SIGILO}" Inexistente Dentre as Opções Permitidas.\nAutomação Encerrada')
  raise SystemExit # se a opção de sigilo não for uma dentre as possíveis, automação encerra o script e apresenta mensagem ao usuário

# fim da checagem da escolha do usuário em relação ao nível de sigilo

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# rotina que pergunta se o sub movimento .1 tb terá o sigilo modificado (por vezes é o movimento que se usa para certidão da juntada dos outros movimentos)



SUBSEQUENCIAL_1_INCLUÍDO = SUBSEQUENCIAL_1_INCLUÍDO.upper()

if (SUBSEQUENCIAL_1_INCLUÍDO !="S" and SUBSEQUENCIAL_1_INCLUÍDO !="N"):SUBSEQUENCIAL_1_INCLUÍDO ="S"
  

if (SUBSEQUENCIAL_1_INCLUÍDO=='S'):
  print(f' A opção escolhida foi no sentido de que o sub movimento {SEQUENCIAL}.1 também terá o sigilo modificado.')
  controleSequencial =0
else:
  print(f' A opção escolhida foi no sentido de que o sub movimento {SEQUENCIAL}.1 NÃO terá o sigilo modificado.')
  controleSequencial =1

# fim da rotina de definição do submovimento .1

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------



#
# Dados informados, automação propriamente iniciada utilizando Selenium
#





driver.get (r'https://projudi2.tjpr.jus.br/projudi/usuario/logon.do?actionType=inicio&r=0.5366016759244643')
time.sleep(1)

  
#login e senha

try:
  elem = driver.find_element_by_xpath("//*[@id='login']")
  elem.clear()
  elem.send_keys(LOGIN)

  elem = driver.find_element_by_xpath("//*[@id=\'senha\']")
  elem.clear()
  elem.send_keys(SENHA)

  elem = driver.find_element_by_xpath("//*[@id='btEntrar']")
  elem.click()
  print('Login no Sistema Projudi em andamento.\nInformando Login e Senha')
except:
  print("Site Projudi Iniciado mas com problemas.\nTente Novamente mais tarde.\nAutomação Encerrada")
  driver.quit()

time.sleep(1)
print('Login no Sistema Projudi efetuado com sucesso')

try:
  elem = driver.find_element_by_xpath("//a[contains(text(),'" + VARA_JUIZO + "')]")
  elem.click()
except:
  print("Vara ou Juízo não Encontrado ou Desnecessário")

time.sleep(1)

driver.get ("https://projudi2.tjpr.jus.br/projudi/processo/buscaProcesso.do?actionType=iniciarSimples")

i = 0


# rotina que localiza formulário de busca simples no Projudi, se não encontrar encerra o script

for x in range(6):
    try:
        elemBuscarProcessoSimplesForm = driver.find_element_by_id('buscaProcessoForm')
        print(f'Formulário de Busca de Processo Encontrado na {x+1}ª tentativa')
        controle=0
        break
    except:
        time.sleep(1)
        controle = 1
        
if controle >0: 
    print('Elemento HTML Não Encontrado.\n Automação encerrada')
    driver.quit()

# fim da rotina de controle do formulário de busca simples do processo

# form de buscar processo na tela, achar o campo e descarregar o conteúdo do número de processo informado na variável PROCESSO_NÚMERO

elem = driver.find_element_by_id('numeroProcesso')
elem.clear()
elem.send_keys(PROCESSO_NÚMERO)
time.sleep(1)

elemBuscarProcessoSimplesForm.submit() # submit no formulário = clicar o enter
time.sleep(1)
print(f'Processo Número: {PROCESSO_NÚMERO} localizado, interação com modificações de sigilo iniciada')


# rotina que vai localizar o sequencial informado pelo usuário - checar se existe

SEQUENCIAL =str(SEQUENCIAL) # converter integer to string 


for x in range(6):
    try:
        elemSequencial = driver.find_element_by_xpath("(.//td[not(contains(text(),':'))][contains(text(),'" + SEQUENCIAL + "')]/..//b[1]//a[1])")
        print(f'Sequencial {SEQUENCIAL} encontrado na {x+1}ª tentativa')
        controle=0
        break
    except:
        time.sleep(1)
        controle = 1
        
if controle >0: 
    print(f'Sequencial {SEQUENCIAL} Não Encontrado. \nAutomação encerrada')
    driver.quit()

 # fim da rotina que checa existência do sequencial



# início do loop para modificaçao do siglo nos sequenciais
 
elemSequencial.click()


# pegando lista de elementos html para Alterar Nível de Sigilo
time.sleep(1)
try:
  elemSequencialSigilo = driver.find_elements_by_partial_link_text("Alterar Nível do Sigilo")
  print('Lista dos Elementos HTML para modificação de sigilo localizada' )
  tamanho_array_elementos =len(elemSequencialSigilo)
except:
  print(f'Lista dos Elementos HTML não localizada. \nAutomação encerrada')
  driver.quit()


# - loop modificação efetiva do sigilo com função auxiliar

#
# função auxiliar
#

def sigilomodificacao(contador):
  global NIVEL_SIGILO
  try:
      elemselect = Select(driver.find_element_by_id('idNivelSigilo'))
      elemselect.select_by_visible_text(NIVEL_SIGILO)
  except:
      print(f'Erro em Selecionar Elemento{SEQUENCIAL}.{contador+1} para Modificação para {NIVEL_SIGILO}.\nAutomação Encerrada.')
      driver.quit()

  time.sleep(1)
  elem = driver.find_element_by_id('saveButton')
  elem.click()
  try:
    driver.switch_to.alert.accept() # qdo se trata de redução do nível de sigilo não existe alerta do javascript
  except:
    pass
  print(f'Modificação do Sigilo Documento Sequencial {SEQUENCIAL}.{contador+1} efetivada para {NIVEL_SIGILO}')
  
 
#fim da função auxiliar
     

#
# Loop de modificação do Sigilo Documentos 
#

contador=0
for item in range(tamanho_array_elementos):
    time.sleep(1)
    if contador >= controleSequencial:
      elemSequencialSigilo = driver.find_elements_by_partial_link_text("Alterar Nível do Sigilo")
      elemSequencialSigilo[contador].click()
      sigilomodificacao(contador)
    contador = contador +1


# fim do loop 
elem = driver.find_element_by_id('backButton')
elem.click()
time.sleep(1)

print(' ')
print(' ')
print(f'Modificação do sigilo dos documentos para {NIVEL_SIGILO}, no Processo Número {PROCESSO_NÚMERO}, finalizada com sucesso.')

driver.quit()

